## Создание умной галереи с помощью распознавания лиц Vision Mail.ru

#### Базовый пример: отправка 1 запроса в Vision API

План:
1. Напишем функцию для отправки запроса
2. Напишем функцию для обработки ответа
3. Прочитаем картинку с диска
4. Сделаем запрос, выведем результат

In [ ]:
import json
import requests
import pprint as pp

<Текст от Юлианы про REST API...>
* Устройство API
* Виды запросов
* Ссылка на документацию

In [ ]:
token = "YOUR_SERVICE_TOKEN_HERE"

HOST = "https://smarty.mail.ru/api/v1/persons/recognize"

In [ ]:
def request_backend(img, token):
    files = {}
    name = "file_0"
    files[name] = img
        
    meta = {
            "space":"0",
            "images": [{"name": name}]
           }

    data = {'meta': json.dumps(meta)}
    response = requests.post(HOST + "?oauth_provider=mcs&oauth_token={}" \
                      .format(token), data=data, files=files)

    return response

<Текст про чтение изображений и вставку их в запрос>

In [ ]:
image_filename = "test.png"

In [ ]:
img = open(image_filename, 'rb')
r = request_backend(img, token)
img.close()

In [ ]:
pp.pprint(r.json())

<Текст про OpenCV>

Чтение и отрисовка картинки

In [ ]:
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image = cv2.imread(image_filename)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(20, 5))
plt.imshow(image, interpolation="lanczos")
plt.axis('off')
plt.show()

In [ ]:
def numpy_to_binary(image):
    return cv2.imencode('.jpg', image)[1].tostring()

In [ ]:
r = request_backend(numpy_to_binary(image), token)
pp.pprint(r.json())

#### Пример: создание умной галереи из пользовательских фотографий

Сделаем следующую задачу: найдем всех уникальных персон в галерее и "разложим" фотографии так, чтобы можно было увидеть все фотографии с ними.

План:
1. Имеется набор фотографий с людьми
2. Построим базу "персон" по фотографиям с помощью Vision API
3. Отсортируем полученные персоны по числу появлений на фотографиях
4. Отрисуем результаты

1. Посмотрим на данные

In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt

In [ ]:
image_names = glob.glob("./got/*.*")
images = [cv2.cvtColor(cv2.imread(name), cv2.COLOR_BGR2RGB) for name in image_names]
print("Total {} images".format(len(images)))

In [ ]:
num_to_show = 20
n_cols = 4
n_rows = num_to_show // n_cols

plt.figure(figsize=(25, n_rows * 5))
for i, image in enumerate(images[:num_to_show], 1):
    plt.subplot(n_rows, n_cols, i)
    plt.imshow(image, interpolation="lanczos")
    plt.axis("off")
plt.tight_layout()
plt.show()

Скорректируем функцию для отправки запроса в Vision API: теперь необходимо установить флаг `"create_new": True` для построения "базы персон" (по умолчанию этого не происходит).

In [ ]:
def request_backend(img, token):
    files = {}
    name = "file_0"
    files[name] = img
        
    meta = {
            "space":"4",
            "create_new": True,
            "images": [{"name": name}],
           }
                                                              
    data = {'meta': json.dumps(meta)}
    response = requests.post(HOST + "?oauth_provider=mcs&oauth_token={}" \
                      .format(token), data=data, files=files)

    return response

Также напишем функцию для обработки ответа, которая будет забирать из него имя найденной персоны (`person1`, `person2`, ...) и координаты лица.

In [ ]:
def parse_response(response):
    json_data = r.json()
    objects = json_data['body']['objects']
    tags = []
    bboxes = []
    
    for obj in objects:
        persons = obj['persons']
        for person in persons:
            tags.append(person['tag'])
            bboxes.append(person['coord'])
        
    return tags, bboxes

In [ ]:
r = request_backend(numpy_to_binary(images[0]), token)
tags, bboxes = parse_response(r)
print(tags)
print(bboxes)

Теперь построим индекс по всем фотографиям в галерее:
1. Каждую фотографию отправим в `Vision API`, получим имена персон и координаты лиц
2. Положим их в словарь вида `{"person1" : [(image_1, bbox_1), (image_2, bbox_2), ...], "person2": [...], ...}`

In [ ]:
import tqdm
from collections import defaultdict

In [ ]:
person_index = defaultdict(list)

In [ ]:
for i, image in enumerate(tqdm.tqdm(images)):
    r = request_backend(numpy_to_binary(image), token)
    persons, bboxes = parse_response(r)
    for person, bbox in zip(persons, bboxes):
        person_index[person].append({"image_idx": i, 
                                     "bbox": bbox})

In [ ]:
pp.pprint(list(person_index.items())[0])

Теперь отрисуем полученных персон в порядке убывания числа фотографий с ними (возьмем первые 10):

In [ ]:
def draw_bbox(image, bbox):
    x1, y1, x2, y2 = bbox
    image_copy = image.copy()
    image_copy = cv2.rectangle(image_copy, (x1, y1), (x2, y2), (0, 255, 127), 8)
    return image_copy

In [ ]:
num_to_show = 5

for i, (person, person_results) in enumerate(sorted(person_index.items(), key=lambda x: len(x[1]), reverse=True)[:num_to_show]):
    
    print("= = = ", person, "= = =")
    
    num_cols = 4
    num_rows = 1 + len(person_results) // num_cols
    plt.figure(figsize=(20, 4 * num_rows))    
    for j, result in enumerate(person_results, 1):
        i = result["image_idx"]
        bbox = result["bbox"]
        image = images[i]
        image = draw_bbox(image, bbox)
        plt.subplot(num_rows, num_cols, j)
        plt.imshow(image, interpolation="gaussian")
        plt.axis("off")
    plt.tight_layout()
    plt.show()